# 여러 이미지 파일을 하나의 데이터셋으로 만들기

이 노트북은 특정 폴더에 있는 여러 이미지 파일을 한 번에 불러와 머신러닝 모델에 사용하기 적합한 형태로 가공하고, 이를 하나의 파일로 저장하는 과정을 다룹니다. 

**주요 과정:**
1.  특정 폴더의 모든 이미지 파일 경로를 가져옵니다.
2.  각 이미지의 크기를 동일하게 맞춥니다 (**리사이즈, Resize**).
3.  이미지를 `Numpy` 배열로 변환합니다.
4.  처리된 모든 이미지 배열을 하나의 압축 파일( `.npz` )로 저장합니다.
5.  저장된 파일을 다시 불러와 이미지가 잘 저장되었는지 시각적으로 확인합니다.

### 1. 라이브러리 임포트

In [ ]:
import PIL.Image as pilimg
import os
import matplotlib.pyplot as plt
import numpy as np

### 2. 이미지 파일 목록 가져오기

`os.listdir()` 함수를 사용하여 지정된 폴더 내의 모든 파일 및 디렉토리 이름을 리스트로 가져옵니다.

In [ ]:
path = "./images/animal"

try:
    filename_list = os.listdir(path)
    print(f"{path} 폴더의 파일 목록:")
    print(filename_list)
except FileNotFoundError:
    print(f"오류: {path} 폴더를 찾을 수 없습니다.")
    print("노트북 파일과 같은 위치에 'images/animal' 폴더를 만들고 이미지 파일들을 넣어주세요.")
    filename_list = []

### 3. 이미지 불러오기, 리사이즈 및 Numpy 배열 변환

가져온 파일 목록을 순회하면서 각 이미지를 처리합니다.

**리사이즈(Resize)의 중요성**: 머신러닝, 특히 딥러닝 모델은 일반적으로 고정된 크기의 입력을 받습니다. 따라서 원본 이미지들의 크기가 제각각이더라도, 모델에 입력하기 전에 모두 동일한 크기(예: 80x80)로 통일시켜 주어야 합니다.

In [ ]:
image_list = []
if filename_list:
    for filename in filename_list:
        filepath = os.path.join(path, filename) # os.path.join으로 안전하게 경로 결합
        try:
            # 이미지 열기
            temp_img = pilimg.open(filepath)
            
            # 이미지 크기를 80x80으로 통일
            resized_img = temp_img.resize((80, 80))
            
            # Numpy 배열로 변환하여 리스트에 추가
            img_array = np.array(resized_img)
            image_list.append(img_array)
            
            print(f"{filename} 처리 완료, 배열 형태: {img_array.shape}")
        except Exception as e:
            print(f"{filename} 처리 중 오류 발생: {e}")
            
    # 최종적으로 리스트에 저장된 이미지 배열의 개수 확인
    print(f"\n총 {len(image_list)}개의 이미지가 처리되었습니다.")

### 4. 데이터셋을 NPZ 파일로 저장

`np.savez()` 함수를 사용하면 여러 개의 Numpy 배열을 하나의 압축된 파일(`.npz`)로 효율적으로 저장할 수 있습니다. 나중에 이 파일 하나만 불러오면 모든 이미지 데이터를 한 번에 사용할 수 있어 편리합니다.

In [ ]:
if image_list:
    # 'data'라는 키로 image_list를 저장합니다.
    np.savez("animal_dataset.npz", data=image_list)
    print(""animal_dataset.npz" 파일로 성공적으로 저장했습니다.")

### 5. 저장된 데이터셋 로드 및 시각화 확인

저장된 `.npz` 파일을 `np.load()`로 다시 불러온 후, `matplotlib`을 사용하여 이미지가 올바르게 저장되었는지 직접 눈으로 확인합니다.

In [ ]:
try:
    # npz 파일 로드
    loaded_data = np.load("animal_dataset.npz")
    
    # 저장할 때 사용한 키('data')로 데이터에 접근
    image_dataset = loaded_data["data"]
    
    print("로드된 데이터셋 형태:", image_dataset.shape)
    
    # Matplotlib를 사용하여 이미지 출력
    plt.figure(figsize=(20, 5)) # 전체 그림의 크기 설정
    num_images = len(image_dataset)
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1) # 1행, num_images열의 그리드에서 i+1번째 위치
        plt.imshow(image_dataset[i])
        plt.axis('off') # 축 정보 끄기
    plt.show()
except FileNotFoundError:
    print("animal_dataset.npz 파일을 찾을 수 없습니다. 이전 단계가 정상적으로 실행되었는지 확인하세요.")